In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import fileinput

#Fit the code to the screen
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

## Reading data

In [2]:
#Reading the movies data
my_path = !pwd
movies_filename = "/ml-1m/movies.dat"
dfmovies = pd.read_csv(my_path[0]+movies_filename, sep='::', header=None)
dfmovies.columns = ["movieId","title", "genres"]

/home/anat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


In [3]:
#Extract the year from the title
dfmovies["year"] = dfmovies['title'].apply(lambda x: x[x.rfind("(")+1:x.rfind(")")])
dfmovies["name"] = dfmovies['title'].apply(lambda x: x[:x.rfind("(")-1])
del dfmovies['title']
#Get the columns to the original order
cols = dfmovies.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[-2:-1] + cols[1:2]
dfmovies = dfmovies[cols]

In [4]:
#Reading the ratings data
ratings_filename = "/ml-1m/ratings.dat"
dfratings = pd.read_csv(my_path[0]+ratings_filename, index_col=None, sep='::', header=None)
dfratings.columns = ["userId", "movieId", "rating", "timestamp"]

/home/anat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


## Creating genres vector for each movie

In [5]:
#Disable false positive Pandas warning for 'SettingWithCopyWarning'
pd.options.mode.chained_assignment = None

#Adding genres vector to each movie
all_genres = ['Mystery', 'Sci-Fi', 'Crime', 'Drama', 'Animation', 'IMAX', 'Action', 'Comedy', 'Documentary', 'War', 'Romance', 'Horror', 
              'Film-Noir', 'Musical', 'Fantasy', 'Adventure', 'Children', 'Thriller', 'Western']
for genre in all_genres:
    dfmovies[genre] = dfmovies['genres'].apply(lambda x: x.find(genre)>=0).astype(int)

In [20]:
users = (dfratings["userId"]).unique()
genres_pref = []
for userId in users:
    user_ratings = dfratings[dfratings["userId"] == userId]
    movies_total = 0
    geners_rating_score = [0] * len(all_genres)
    for index, rating in user_ratings.iterrows():
        movieId = rating["movieId"]
        score = rating["rating"]
        movie_data = dfmovies.loc[dfmovies["movieId"] == movieId]
        for genre in all_genres:
            if (movie_data[genre].item() == 1):
                geners_rating_score[all_genres.index(genre)] += score
    genres_pref.append(geners_rating_score)
    if (userId % 1000 == 0):
        print userId

1000
2000
3000
4000
5000
6000


In [30]:
#Save the genres prefrence to dataframe and to pickle
user_pref_df = pd.DataFrame(genres_pref)
user_pref_df.columns = all_genres
user_pref_df.index += 1
with open(my_path[0]+'/explore/user_genre_pref.pik', 'wb') as handle:
    pickle.dump(user_pref_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
user_pref_df

Mystery  Sci-Fi  Crime  Drama  Animation  IMAX  Action  Comedy  \
1           0      13      8     93         74     0      21      58   
2          10      61     43    308          0     0     196      89   
3           3      23      0     32         12     0      91     113   
4           0      32      5     25          0     0      79       0   
5          25      46     69    322         16     0      81     191   
6           0       7      6     82         38     0      44     120   
7           4      43      8     32          0     0     125      13   
8          14      58     52    412         17     0     145      70   
9           4      35     60    210         37     0      70     101   
10         29     297     54    485        142     0     317     757   
11         11       7     31    199          1     0      32     266   
12          3       4     22     52          0     0      23      23   
13         12     131     26     85          6     0     224      54   
14          5       7     10     46          0     0      13      23   
15         63      59    103    275          8     0     188     189   
16          4      16      1     13         17     0      16      70   
17         40     465     91    286         45     0     252     143   
18         32     152    114    408        184     0     403     282   
19         27     174     62    149        110     0     361     287   
20          9      36     21     26          0     0      71      15   
21          1      20      1      1         43     0      20      21   
22         28     171     77    194         39     0     287     446   
23         47     388    181    232         44     0     348     306   
24          3      49     56    267         16     0     106     283   
25          3     160      9     74         29     0     174      99   
26         36      96     37    472         33     0     204     597   
27         19      16     29    109          8     0      25     105   
28         18      54     25    178         11     0      74     137   
29          3     102     30     88          4     0     319      39   
30          8      13     21     94          3     0      15      27   
...       ...     ...    ...    ...        ...   ...     ...     ...   
6011       20     301     32    242         37     0     376     172   
6012        4       2      5     47          3     0      16       5   
6013        5      20      1    255         58     0      27     164   
6014       31      90     24    132         33     0      63      84   
6015        4       0      7     33         72     0       4     156   
6016       98     207    234   1422         78     0     455    1132   
6017        6      10      6     42          5     0      30      31   
6018       17     272     19    179         20     0     338      64   
6019       19      84     31     56          2     0     148      81   
6020        5      40     26     41          0     0     116      22   
6021       11     123     28    121         41     0     218     172   
6022        0      40     19     59         15     0      69      66   
6023       18     109    133    124          5     0     439      58   
6024        9     106     32    108         12     0     273     101   
6025       33      63     43    256         30     0     100     448   
6026        5     221     17     78          3     0     116      41   
6027        5      28      3     44          4     0      80      63   
6028        0      36     27     48          4     0     145       0   
6029        0       6     10     58          7     0      14      39   
6030       14     349      8     58         29     0     219      84   
6031        5      41      9     64          8     0      56      58   
6032       24      31     16    158         43     0      40     179   
6033        0      65     23     75          3     0     192      24   
6034       23       5     36     12          3